In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import time

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Create Dataset of images and labels from CSV
class ImageDataset(Dataset):
    def __init__(self, csvdata, transform):
        self.dataset = list(csvdata.values)
        self.transform = transform

        labels = []
        images = []

        for row in self.dataset:
            # extract label from first column.
            labels.append(row[0])
            images.append(row[1:])
        self.labels = np.asarray(labels)
        # Dimension of Image is 28 * 28 * 1
        self.images = np.asarray(images).reshape(-1, 28, 28, 1).astype('float32')

    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

In [ ]:
# Validation Split
validation_split = 0.1 # Percentage of training data to use for validation
batch_size = 64
# Hyperparameters (to optimize)
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 20

In [ ]:
# Load csv using pandas
train_csv = pd.read_csv("./content/data/fashion-mnist_train.csv")
test_csv = pd.read_csv("./content/data/fashion-mnist_test.csv")

# Split training dataset into training and validation
num_train = len(train_csv)
indices = list(range(num_train))
split_idx = int(validation_split * num_train)
train_indices, val_indices = indices[split_idx:], indices[:split_idx]

train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

# Transform data into Tensor that has a range from 0 to 1
transform = transforms.Compose([transforms.ToTensor()])
train_set = ImageDataset(train_csv, transform=transform)
test_set = ImageDataset(test_csv, transform=transform)

# Create Pytorch dataloaders
train_loader = DataLoader(train_set, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(train_set, batch_size=batch_size, sampler=val_sampler)
test_loader = DataLoader(test_set, batch_size=len(train_csv))

In [ ]:
cnn_out_channels = [64, 128, 256, 256, 256]
cnn_filter_size_mat = [[3, 3, 3, 3, 3],
                       [4, 4, 3, 3, 3],
                       [2, 2, 2, 2, 2]]
cnn_filter_sizes = cnn_filter_size_mat[2]
cnn_filter_paddings = [1, 1, 1, 1, 1]
cnn_filter_strides = [1, 1, 1, 1, 1]
cnn_pooling_sizes = [2, 2, 1, 1, 1]
cnn_pooling_strides = [2, 2, 1, 1, 1]
fc_out_features = [128, 10]
dropout_prob = 0.2

image_size = 28
for i in range(len(cnn_out_channels)):
    filter_out_size = (image_size + 2*cnn_filter_paddings[i] - cnn_filter_sizes[i])//cnn_filter_strides[i] + 1
    pooling_out_size = filter_out_size//cnn_pooling_strides[i]
    image_size = pooling_out_size

print(image_size)
image_size=1

10


In [ ]:
# Define the CNN model

class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=cnn_out_channels[0], kernel_size=cnn_filter_sizes[0], padding=cnn_filter_paddings[0], stride=cnn_filter_strides[0]),
            nn.BatchNorm2d(cnn_out_channels[0]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=cnn_pooling_sizes[0], stride=cnn_pooling_strides[0])
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=cnn_out_channels[0], out_channels=cnn_out_channels[1], kernel_size=cnn_filter_sizes[1], padding=cnn_filter_paddings[1], stride=cnn_filter_strides[1]),
            nn.BatchNorm2d(cnn_out_channels[1]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=cnn_pooling_sizes[1], stride=cnn_pooling_strides[1])
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=cnn_out_channels[1], out_channels=cnn_out_channels[2], kernel_size=cnn_filter_sizes[2], padding=cnn_filter_paddings[2], stride=cnn_filter_strides[2]),
            nn.BatchNorm2d(cnn_out_channels[2]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=cnn_pooling_sizes[2], stride=cnn_pooling_strides[2])
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=cnn_out_channels[2], out_channels=cnn_out_channels[3], kernel_size=cnn_filter_sizes[3], padding=cnn_filter_paddings[3], stride=cnn_filter_strides[3]),
            nn.BatchNorm2d(cnn_out_channels[3]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=cnn_pooling_sizes[3], stride=cnn_pooling_strides[3])
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=cnn_out_channels[3], out_channels=cnn_out_channels[4], kernel_size=cnn_filter_sizes[4], padding=cnn_filter_paddings[4], stride=cnn_filter_strides[4]),
            nn.BatchNorm2d(cnn_out_channels[4]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=cnn_pooling_sizes[4], stride=cnn_pooling_strides[4]),
            nn.AdaptiveAvgPool2d(output_size=(1, 1))
        )

        self.fc1 = nn.Linear(in_features=image_size*image_size*cnn_out_channels[4], out_features=fc_out_features[0])
        self.drop = nn.Dropout2d(dropout_prob)
        self.fc2 = nn.Linear(in_features=fc_out_features[0], out_features=fc_out_features[1])

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        return out

    def initialize_weights(self, m, method):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            if method == 'xavier':
                nn.init.xavier_uniform_(m.weight)
            elif method == 'he':
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
            else:  # Default: Random Initialization
                nn.init.normal_(m.weight, mean=0.0, std=0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

model = ConvNet(num_classes).to(device)
model.apply(lambda m: model.initialize_weights(m, 'random'))
# Loss function and optimizer
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training Function
def train(model, data_loader, loss_criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    for batch_idx, (images, labels) in enumerate(data_loader):
        images, labels = images.to(device), labels.to(device)
        # Convert labels to LongTensor
        labels = labels.type(torch.LongTensor).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted_labels = torch.max(outputs, 1) # Get predicted class labels
        total_samples += labels.size(0)
        correct_predictions += (predicted_labels == labels).sum().item()

    epoch_loss = running_loss / len(data_loader)
    epoch_accuracy = correct_predictions / total_samples
    return epoch_loss, epoch_accuracy

# Validation function
def validate(model, data_loader, loss_criterion, device):
    model.eval()  # Set to evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
      for batch_idx, (images, labels) in enumerate(data_loader):
          images, labels = images.to(device), labels.to(device)
          # Convert labels to LongTensor
          labels = labels.type(torch.LongTensor).to(device)  # This line is added

          outputs = model(images)
          loss = loss_criterion(outputs, labels)
          running_loss += loss.item()

          _, predicted_labels = torch.max(outputs, 1)
          total_samples += labels.size(0)
          correct_predictions += (predicted_labels == labels).sum().item()

      epoch_loss = running_loss / len(data_loader)
      epoch_accuracy = correct_predictions / total_samples
      return epoch_loss, epoch_accuracy

# Create Confusion Matrix
def create_conf_matrix(batch_labels, predicted_test_labels):
  conf_matrix = np.zeros((10, 10), dtype=int)
  for i in range(len(batch_labels)):
    conf_matrix[batch_labels[i], predicted_test_labels[i]] += 1
  return conf_matrix

# Test function
def test(model, data_loader, loss_criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(data_loader):
            images, labels = images.to(device), labels.to(device)
            # Convert labels to LongTensor
            labels = labels.type(torch.LongTensor).to(device)

            outputs = model(images)
            loss = loss_criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted_labels = torch.max(outputs, 1)
            total_samples += labels.size(0)
            conf_mat = create_conf_matrix(labels, predicted_labels)
            correct_predictions += (predicted_labels == labels).sum().item()

    test_loss = running_loss / len(data_loader)
    test_accuracy = correct_predictions / total_samples
    return conf_mat, test_loss, test_accuracy

In [ ]:
# Initialize Model, Loss function, and Optimizer
model = ConvNet(num_classes).to(device)
# Different initialization of model weights
model.apply(lambda m: model.initialize_weights(m, 'random'))
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Store losses and accuracies for plotting
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
best_val_loss = np.inf
test_loss_final = 0
test_accuracy_final = 0

# Training loop
for epoch in range(num_epochs):
    train_start = time.time()
    train_loss, train_accuracy = train(model, train_loader, loss_criterion, optimizer, device)
    train_end = time.time()
    val_loss, val_accuracy = validate(model, val_loader, loss_criterion, device)
    val_end = time.time()
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Time: {train_end-train_start}, Val Time: {val_end-train_end}')

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')

    if val_loss < best_val_loss:
      best_val_loss = val_loss
      # Save the best model weights based on validation accuracy here
      torch.save(model.state_dict(), 'best_cnn_val.pth')
      print(f"Validation loss improved! Best validation accuracy so far: {val_accuracy:.2f}, Best validation loss: {best_val_loss: .4f} (Model saved)")

print('Finished Training and Validation')
print(f'Best Validation Loss Achieved: {best_val_loss:.4}')

/home/pankaj/anaconda3/lib/python3.12/site-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch [1/20], Train Time: 6.562299966812134, Val Time: 0.24617600440979004
Epoch [1/20], Train Loss: 0.4916, Train Acc: 0.82, Val Loss: 0.3700, Val Acc: 0.87
Validation loss improved! Best validation accuracy so far: 0.87, Best validation loss:  0.3700 (Model saved)
Epoch [2/20], Train Time: 6.301050662994385, Val Time: 0.24965596199035645
Epoch [2/20], Train Loss: 0.3236, Train Acc: 0.88, Val Loss: 0.4415, Val Acc: 0.85
Epoch [3/20], Train Time: 6.346246004104614, Val Time: 0.24689698219299316
Epoch [3/20], Train Loss: 0.2784, Train Acc: 0.90, Val Loss: 0.2673, Val Acc: 0.91
Validation loss improved! Best validation accuracy so far: 0.91, Best validation loss:  0.2673 (Model saved)
Epoch [4/20], Train Time: 6.230573654174805, Val Time: 0.24828529357910156
Epoch [4/20], Train Loss: 0.2504, Train Acc: 0.91, Val Loss: 0.2879, Val Acc: 0.90
Epoch [5/20], Train Time: 6.152796745300293, Val Time: 0.24851083755493164
Epoch [5/20], Train Loss: 0.2281, Train Acc: 0.92, Val Loss: 0.2927, Val Ac

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Testing after training
# Load the best model saved during validation
best_model = ConvNet(num_classes).to(device)
best_model.load_state_dict(torch.load('best_cnn_val.pth'))
test_start = time.time()
conf_mat, test_loss_final, test_accuracy_final = test(best_model, test_loader, loss_criterion, device)
test_end = time.time()
print(f'Test Loss: {test_loss_final:.4f}, Test Acc: {test_accuracy_final:.4f}, Test time: {test_end-test_start}')

# 5. Plotting Losses and Accuracies
epochs = range(1, num_epochs + 1)

fig = plt.figure(figsize=(12, 5))
# plot_name = f'Kernel Size: {cnn_filter_sizes}, Kernels(Depth): {cnn_out_channels}, Pooling: MaxPool({cnn_pooling_sizes})'
plot_name = f'Kernel Size: {cnn_filter_sizes}, Kernels(Depth): {cnn_out_channels}, Pooling: GlobalPool({cnn_pooling_sizes + [7]})'
fig.suptitle(plot_name)

# Plot Losses
plt.subplot(1, 2, 1) # 1 row, 2 columns, first subplot
plt.plot(epochs, train_losses, 'b-', label='Training Loss')
plt.plot(epochs, val_losses, 'r-', label='Validation Loss')
plt.scatter([num_epochs], [test_loss_final], color='g', marker='o', label='Test Loss (Final Epoch)')
plt.title('Training, Validation, and Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot Accuracies
plt.subplot(1, 2, 2) # 1 row, 2 columns, second subplot
plt.plot(epochs, train_accuracies, 'b-', label='Training Accuracy')
plt.plot(epochs, val_accuracies, 'r-', label='Validation Accuracy')
plt.scatter([num_epochs], [test_accuracy_final], color='g', marker='o', label='Test Accuracy (Final Epoch)')
plt.title('Training, Validation, and Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout() # Adjust subplot params for a tight layout
plt.savefig(f'{plot_name}.png')
plt.show()

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.01 GiB. GPU 0 has a total capacity of 5.68 GiB of which 1.43 GiB is free. Including non-PyTorch memory, this process has 4.22 GiB memory in use. Of the allocated memory 3.48 GiB is allocated by PyTorch, and 613.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import seaborn as sn

desc_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
df_cm = pd.DataFrame(conf_mat, index = [i for i in desc_labels],
                  columns = [i for i in desc_labels])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

plt.savefig(f'{plot_name}_confmat.png')